In [ ]:
from importlib import reload
from time import sleep

import numpy as np
import polars as pl
from plotly import graph_objects as go

import coolsearch.search as csearch
import coolsearch.models as cmodel
import coolsearch.utility_functions as util

reload(csearch)
reload(cmodel)

util.set_plotly_template()

SEED = 137

In [ ]:
from sklearn import datasets, model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn import base

X, y = datasets.make_classification(10000, 10, random_state=SEED)
X_train, X_val, y_train, y_val = model_selection.train_test_split(
    X, y, random_state=SEED)

In [ ]:
clf = DecisionTreeClassifier(random_state=SEED)
clf.fit(X_train, y_train)

print(classification_report(y_val, clf.predict(X_val), digits=4))

In [ ]:
def clf_val(clf, param_range: dict, param_types: dict | None, X_train, X_val, Y_train, Y_val, score, fixed_params={}, seed=None):
    """Create search obj? fixed params?

    Note: prefer deterministic classifier (or provide seed)?"""

    def objective(**kwargs):
        clf.set_params(**kwargs)
        clf.fit(X_train, Y_train)

        pred_val = clf.predict(X_val)

        return score(Y_val, pred_val)

    return csearch.CoolSearch(objective, param_range, param_types, fixed_params)

In [ ]:
param_range = dict(
    max_depth=(1, 30),
    min_samples_leaf=(1, 50)
)
param_types = dict.fromkeys(param_range.keys(), "int")

search = clf_val(DecisionTreeClassifier(random_state=SEED), param_range, param_types, X_train,
                    X_val, y_train, y_val, score=accuracy_score)

print(search)


In [ ]:
_ = search.grid_search(20)

In [ ]:
search.samples.sort(-pl.col("score"))[0,:]

In [ ]:
s = search.samples

go.Figure(
    go.Scatter(x=s["max_depth"], y=s["min_samples_leaf"],
               marker_color=(s["score"]), mode="markers")
)

In [ ]:
marg = search.marginals()


for k in marg.keys():
    go.Figure(
        go.Scatter(x=marg[k][k],
                   y=marg[k]["mean"])
    ).update_layout(title=k).show()